# TASK #1: UNDERSTAND THE PROBLEM STATEMENT AND BUSINESS CASE  

![alt text](https://drive.google.com/uc?id=1mVeeFi7cusk7ljpMf0cSSjLvi0Z3gaEY)

![alt text](https://drive.google.com/uc?id=1gYn6Dyi7mL1RDbZQzTKuirkw7GnbWwK6)

# TASK #2: IMPORT MODEL WITH PRE-TRAINED WEIGHTS

In [ ]:

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random 
import os
import PIL.Image
import cv2
import random
from PIL import Image # Python Image Library is a library that adds support for opening, manipulating, and saving many different
                      # image file formats

In [ ]:
tf.__version__

In [ ]:
# Loading trained inceptionNet model, imagenet (include_top=False)
base_model = tf.keras.applications.InceptionV3(include_top = True, weights = 'imagenet')

Loading trained inceptionNet model, imagenet (include_top=True) <br>
base_model = tf.keras.applications.InceptionV3(include_top = <b>True</b>, weights = 'imagenet')

MINI CHALLENGE #1: 
- How many total parameters exist in inceptionNet V3?
- Set include top = True and indicate how many total parameters exist now.

# TASK #3: GET AN IMAGE AND PRE-PROCESS IT

In [ ]:
# Open the first image
# Source: https://www.pxfuel.com/en/free-photo-xxgfs
img_1 = Image.open("/content/drive/My Drive/Colab Notebooks/Modern AI Portfolio Builder/Art Creation by AI/mars.jpg")

# Open the second image
# Source: https://commons.wikimedia.org/wiki/File:Georges_Garen_embrasement_tour_Eiffel.jpg
img_2 = Image.open('/content/drive/My Drive/Colab Notebooks/Modern AI Portfolio Builder/Art Creation by AI/eiffel.jpg')

# Blend the two images

image = Image.blend(img_1, img_2, 0.5) # alpha --> The interpolation alpha factor. If alpha is 0.0, a copy of the first image is returned.
# If alpha is 1.0, a copy of the second image is returned. 

# Save the blended image
image.save("img_0.jpg")

In [ ]:
# Load the image
Sample_Image = tf.keras.preprocessing.image.load_img('img_0.jpg')

In [ ]:
Sample_Image

In [ ]:
# Get the shape of the image
np.shape(Sample_Image)

In [ ]:
# Check out the type of the image
type(Sample_Image)

In [ ]:
# Convert to numpy array
Sample_Image = tf.keras.preprocessing.image.img_to_array(Sample_Image)

# Sample_Image = np.array(Sample_Image)

In [ ]:
# Confirm that the image is converted to Numpy array
type(Sample_Image)

In [ ]:
# Obtain the max and min values
print('min pixel values = {}, max pixel values = {}'.format(Sample_Image.min(), Sample_Image.max()))

In [ ]:
# Normalize the input image
Sample_Image = np.array(Sample_Image)/255.0
Sample_Image.shape

In [ ]:
# Let's verify normalized images values!
print('min pixel values = {}, max pixel values = {}'.format(Sample_Image.min(), Sample_Image.max()))

In [ ]:
Sample_Image = tf.expand_dims(Sample_Image, axis = 0)

In [ ]:
np.shape(Sample_Image)

MINI CHALLENGE #2: 
- Perform the opposite of expand dimension 
- Plot the image

# TASK #4: RUN THE PRETRAINED MODEL AND EXPLORE ACTIVATIONS

# NOTES:
- Select a layer and attempt at maximizing the loss which is the activations generated by the layer of interest.
- We can select any layer we choose, early layers generate simple features such as edges and deep layers generate more complex features such as entire face, car or tree.
- Inception network has multiple concatenated layers named 'mixed' 

In [ ]:
base_model.summary()

In [ ]:
# Maximize the activations of these layers

names = ['mixed3', 'mixed5', 'mixed7']

# names = ['mixed3']

layers = [base_model.get_layer(name).output for name in names]

# Create the feature extraction model
deepdream_model = tf.keras.Model(inputs = base_model.input, outputs = layers)

In [ ]:
deepdream_model.summary()

In [ ]:
# Let's run the model by feeding in our input image and taking a look at the activations "Neuron outputs"
activations = deepdream_model(Sample_Image)
activations

In [ ]:
len(activations)

MINI CHALLENGE #3: 
- Generate the activations for a deeper layer such as 'mixed8' and 'mixed9'.
- What is the size of the generated activations?
- Combine 4 activations from early and deeper layers such as 'mixed3', 'mixed5', 'mixed8', 'mixed9'


# TASK #5: UNDERSTAND HOW DEEP DREAM ALGORITHM WORKS 

![alt text](https://drive.google.com/uc?id=1pbBZWp8JCgIWBzrXhDoQZSi5ekeR2Sga)

Source #1: https://www.topbots.com/advanced-topics-deep-convolutional-neural-networks/

Source #2: https://wccftech.com/nvidia-demo-skynet-gtc-2014-neural-net-based-machine-learning-intelligence/

![alt text](https://drive.google.com/uc?id=1mh4OUKw5cFjxFa7zW-JZyLwHAQHRIW_-)

![alt text](https://drive.google.com/uc?id=1k1Hp6M1GB9qCaiPPAJQgGwTh5VNzEm4l)

![alt text](https://drive.google.com/uc?id=1usz7K6Bmo_9u7n56NV3PCvLBRgSN1TkQ)

![alt text](https://drive.google.com/uc?id=1R7_C4r4vy2tqIB5Pi-ltyY2N_WC6jSYF)

# TASK #6: UNDERSTAND HOW TO PERFORM GRADIENT CALCULATION AND TF.GRADIENTTAPE()

- tf.GradientTape() is used to record operations for automatic differentiation
- For example, Let's assume we have the following functions y = x^3. 
- The gradient at x = 2 can be computed as follows: dy_dx = 3 * x^2 = 3 * 2^2 = 12. 


In [ ]:
x = tf.constant(2.0)

In [ ]:
with tf.GradientTape() as g:
  g.watch(x)
  y = x * x * x
dy_dx = g.gradient(y, x) # Will compute to 12

In [ ]:
dy_dx

MINI CHALLENGE #4: 
- Using tf.GradientTape(), calculate the gradient of y = x^4 + x^5 at x = 5
- Verify your answer by manually differentation the equation

In [ ]:
x = tf.constant(5.0)

In [ ]:
with tf.GradientTape() as g:
  g.watch(x)
  y = (x * x * x * x) + (x * x * x * x * x)  
dy_dx = g.gradient(y, x) 
dy_dx

# TASK #7: IMPLEMENT DEEP DREAM ALGORITHM - STEP #1 LOSS CALCULATION

- CREDITS: The DeepDream Code has been adopted from Keras Documentation:
- https://www.tensorflow.org/tutorials/generative/deepdream

In [ ]:
# Since the cal_closs function includes expand dimension, let's squeeze the image (reduce_dims)
Sample_Image.shape

In [ ]:
Sample_Image = tf.squeeze(Sample_Image, axis = 0)

In [ ]:
Sample_Image.shape

In [ ]:
def calc_loss(image, model):
# Function used for loss calculations
# It works by feedforwarding the input image through the network and generate activations
# Then obtain the average and sum of those outputs

  img_batch = tf.expand_dims(image, axis=0) # Convert into batch format
  layer_activations = model(img_batch) # Run the model
  print('ACTIVATION VALUES (LAYER OUTPUT) =\n', layer_activations)
  # print('ACTIVATION SHAPE =\n', np.shape(layer_activations))

  losses = [] # accumulator to hold all the losses
  for act in layer_activations:
    loss = tf.math.reduce_mean(act) # calculate mean of each activation 
    losses.append(loss)
  
  print('LOSSES (FROM MULTIPLE ACTIVATION LAYERS) = ', losses)
  print('LOSSES SHAPE (FROM MULTIPLE ACTIVATION LAYERS) = ', np.shape(losses))
  print('SUM OF ALL LOSSES (FROM ALL SELECTED LAYERS)= ', tf.reduce_sum(losses))

  return  tf.reduce_sum(losses) # Calculate sum 

In [ ]:
loss = calc_loss(tf.Variable(Sample_Image), deepdream_model)

In [ ]:
loss # Sum up the losses from both activations

MINI CHALLENGE #5: 
- What is the sum of all losses when 'mixed3' layer is the only layer used for activations generation?

# TASK #8: IMPLEMENT DEEP DREAM ALGORITHM - STEP #2 (CALCULATE THE GRADIENT)

- In this step, we will rely on the loss that has been calculated in the previous step and calculate the gradient with respect to the given input image and then add it to the input original image. 
- Doing so iteratively will result in feeding images that continiously and increasingly excite the neurons and generate more dreamy like images!


In [ ]:
# When you annotate a function with tf.function, the function can be called like any other python defined function. 
# The benefit is that it will be compiled into a graph so it will be much faster and could be executed over TPU/GPU

@tf.function
def deepdream(model, image, step_size):
    with tf.GradientTape() as tape:
      # This needs gradients relative to `img`
      # `GradientTape` only watches `tf.Variable`s by default
      tape.watch(image)
      loss = calc_loss(image, model) # call the function that calculate the loss 

    # Calculate the gradient of the loss with respect to the pixels of the input image.
    # The syntax is as follows: dy_dx = g.gradient(y, x) 
    gradients = tape.gradient(loss, image)

    print('GRADIENTS =\n', gradients)
    print('GRADIENTS SHAPE =\n', np.shape(gradients))

    # tf.math.reduce_std computes the standard deviation of elements across dimensions of a tensor
    gradients /= tf.math.reduce_std(gradients)  

    # In gradient ascent, the "loss" is maximized so that the input image increasingly "excites" the layers.
    # You can update the image by directly adding the gradients (because they're the same shape!)
    image = image + gradients * step_size
    image = tf.clip_by_value(image, -1, 1)

    return loss, image

In [ ]:
def run_deep_dream_simple(model, image, steps = 100, step_size = 0.01):
  # Convert from uint8 to the range expected by the model.
  image = tf.keras.applications.inception_v3.preprocess_input(image)

  for step in range(steps):
    loss, image = deepdream(model, image, step_size)
    
    if step % 100 == 0:
      plt.figure(figsize=(12,12))
      plt.imshow(deprocess(image))
      plt.show()
      print ("Step {}, loss {}".format(step, loss))

  # clear_output(wait=True)
  plt.figure(figsize=(12,12))
  plt.imshow(deprocess(image))
  plt.show()

  return deprocess(image)

In [ ]:
def deprocess(image):
  image = 255*(image + 1.0)/2.0
  return tf.cast(image, tf.uint8)


In [ ]:
Sample_Image.shape

In [ ]:
# Let's Load the image again and convert it to Numpy array 
Sample_Image = np.array(tf.keras.preprocessing.image.load_img('img_0.jpg'))
dream_img = run_deep_dream_simple(model = deepdream_model, image = Sample_Image, steps = 4000, step_size = 0.001)


# TASK #9: (VIDEO) APPLY DEEPDREAM TO GENERATE A SERIES OF IMAGES

In [ ]:
image = tf.keras.preprocessing.image.load_img("img_0.jpg")

In [ ]:
plt.imshow(image)

In [ ]:
# Name of the folder
dream_name = 'mars_eiffel'

In [ ]:
# Blended image dimension

x_size = 910 # larger the image longer is going to take to fetch the frames 
y_size = 605

In [ ]:
# Define Counters 
created_count = 0
max_count = 50

In [ ]:
# This helper function loads an image and returns it as a numpy array of floating points

def load_image(filename):
    image = PIL.Image.open(filename)
    return np.float32(image)

In [ ]:
for i in range(0, 50):
    # Make sure to create a new folder entitled 'mars_eiffel' and place img_0 in it
    # Get into the dream directory and look for the number of images and then figure out what is the latest image. Hence this 
    # image we are going to start with and let it dream on and on
     
    if os.path.isfile('/content/drive/My Drive/Colab Notebooks/Modern AI Portfolio Builder/Art Creation by AI/{}/img_{}.jpg'.format(dream_name, i+1)):
        print("{} present already, continue fetching the frames...".format(i+1))
        
    else:
        # Call the load image funtion
        img_result = load_image(r'/content/drive/My Drive/Colab Notebooks/Modern AI Portfolio Builder/Art Creation by AI/{}/img_{}.jpg'.format(dream_name, i))

    
        # Zoom the image 
        x_zoom = 2 # this indicates how quick the zoom is 
        y_zoom = 1
        
        # Chop off the edges of the image and resize the image back to the original shape. This gives the visual changes of a zoom
        img_result = img_result[0+x_zoom : y_size-y_zoom, 0+y_zoom : x_size-x_zoom]
        img_result = cv2.resize(img_result, (x_size, y_size))
        
        # Adjust the RGB value of the image
        img_result[:, :, 0] += 2  # red
        img_result[:, :, 1] += 2  # green
        img_result[:, :, 2] += 2  # blue
        
        # Deep dream model  
        img_result = run_deep_dream_simple(model = deepdream_model, image = img_result, steps = 500, step_size = 0.001)
        
        # Clip the image, convert the datatype of the array, and then convert to an actual image. 
        img_result = np.clip(img_result, 0.0, 255.0)
        img_result = img_result.astype(np.uint8)
        result = PIL.Image.fromarray(img_result, mode='RGB')
        
        # Save all the frames in the dream location
        result.save(r'/content/drive/My Drive/Colab Notebooks/Modern AI Portfolio Builder/Art Creation by AI/{}/img_{}.jpg'.format(dream_name, i+1))
        
        created_count += 1
        if created_count > max_count:
            break

# TASK #10: (VIDEO) CREATE A VIDEO FROM ALL THE FRAMES

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Unzip the folder

from zipfile import ZipFile
file_name = "mars_eiffel.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
# Path of all the frames

dream_path = 'mars_eiffel'

In [ ]:
# Define the codec and create VideoWriter object 
# Download FFmeg 

fourcc = cv2.VideoWriter_fourcc(*'XVID') # FourCC is a 4-byte code used to specify the video codec

out = cv2.VideoWriter('deepdreamvideo.avi', fourcc , 5.0, (910, 605)) # Specify the fourCC, frames per second (fps),
                                                                            # and frame size
# The frames per second value is depends on few important things
# 1. The number of frames we have created. Less number of frames brings small fps
# 2. The larger the image the bigger the fps value. For example, 1080 pixel image can bring 60 fps 

In [ ]:
for i in range(9999999999999):
    
    # Get into the dream directory and looks for the number of images and then figure out what is the latest image. Hence with 
    # this image we are going to start with and let it dream on and on
    if os.path.isfile('mars_eiffel/img_{}.jpg'.format(i+1)):
        pass
    # Figure out how long the dream is 
    else:
        dream_length = i
        break

In [ ]:
dream_length

In [ ]:
for i in range(dream_length):
    
    # Build the frames of cv2.VideoWriter
    img_path = os.path.join(dream_path,'img_{}.jpg'.format(i)) # join the dream path
    
    print(img_path) # print the image path 
    
    frame = cv2.imread(img_path)
    out.write(frame)

out.release()

# GREAT JOB!

# MINI CHALLENGE SOLUTIONS

MINI CHALLENGE #1: 
- How many total parameters exist in inceptionNet V3?
- Set include top = True and indicate how many total parameters exist now.

In [ ]:
Total Params = 23,851,784 (include Top = True)
Total params: 21,802,784 (include Top = False)


MINI CHALLENGE #2: 

In [ ]:
plt.imshow(np.squeeze(Sample_Image))

MINI CHALLENGE #3: 
- Generate the activations for a deeper layer such as 'mixed8' and 'mixed9'.
- What is the size of the generated activations?
- Combine 4 activations from early and deeper layers such as 'mixed3', 'mixed5', 'mixed8', 'mixed9'

In [ ]:
names = ['mixed8', 'mixed9']
# shape=(1, 13, 21, 1280)
names = ['mixed3', 'mixed5', 'mixed8', 'mixed9']

MINI CHALLENGE #4: 
- Using tf.GradientTape(), calculate the gradient of y = x^4 + x^5 at x = 5
- Verify your answer by manually differentation the equation

In [ ]:
x = tf.constant(5.0)
with tf.GradientTape() as g:
  g.watch(x)
  y = (x * x * x * x) + (x * x * x * x * x)
dy_dx = g.gradient(y, x) # Will compute to 12
dy_dx

In [ ]:
# 500+3125 = 3625

MINI CHALLENGE #5: 
- What is the sum of all losses when 'mixed3' layer is the only layer used for activations generation?

In [ ]:
# <tf.Tensor: shape=(), dtype=float32, numpy=0.36584973>